In [1]:
# VYTVORENIE DATASETOV detached_curves.pkl, detached_curves_samples_knn.pkl, detached_curves_samples_svr.pkl

In [3]:
# BLOK 1
# Importovanie kniznic.

import numpy as np
import pandas as pd
import sqlite3
import io

import random

In [2]:
# BLOK 2
# Funkcia pre vyber svetelnej krivky z nespracovanych dat. Zadefinovanie cesty k databaze (subor .db).

def get_curve(txt):
    out = io.BytesIO(txt)
    out.seek(0)
    return np.load(out)

FILEPATH = '../data-upjs/detached.db'

In [3]:
# BLOK 3 
# Vytvorenie pripojenia na subor. Vypis vsetkych tabuliek v databaze.

conn = sqlite3.connect(FILEPATH)

sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor = conn.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('parameters',), ('curves',), ('auxiliary',)]


In [23]:
# BLOK 4
# Nacitanie parametrov.

df_parameters = pd.read_sql_query("SELECT id, mass_ratio, primary__surface_potential, secondary__surface_potential, primary__t_eff, secondary__t_eff, inclination FROM parameters", conn)

In [ ]:
# BLOK 5
# Nacitanie id kriviek po castiach, ulozenie id do dataframu, vyber nahodnej vzorky v pocte 100 000
ids = []  
for chunk in pd.read_sql("SELECT id FROM curves", conn, chunksize=10000):
    ids.append(chunk)
    
df_ids = pd.DataFrame()
df_ids = pd.concat(ids, ignore_index=True)

ids_sample = random.sample(list(df_ids['id']), 100000)

In [17]:
# BLOK 6
# Nacitanie konkretnych svetelnych kriviek pomocou id
curves_from_db = []
for id in ids_sample:
    curves_from_db.append(pd.read_sql_query(f"SELECT * FROM curves WHERE id = {id}", conn))
    
df_curves = pd.DataFrame()
df_curves = pd.concat(curves_from_db, ignore_index=True)

In [24]:
# BLOK 7
# Vytvorenie tabulky. Jeden riadok = jeden system zakrytovych premennych hviezd v 13 filtroch.

df1=df_parameters[["id","primary__t_eff", "secondary__t_eff", "mass_ratio", "primary__surface_potential",
                     "secondary__surface_potential", "inclination"]]
df2=df_curves[["id", "Bessell_U", "Bessell_B", "Bessell_V", "Bessell_R", "Bessell_I", "SLOAN_u", "SLOAN_g", "SLOAN_r", "SLOAN_i", "SLOAN_z", "Kepler", "GaiaDR2", "TESS"]]

df_merged = pd.merge(df1,df2,on="id")
df_merged["Bessell_U"] = df_merged["Bessell_U"].apply(get_curve)
df_merged["Bessell_V"] = df_merged["Bessell_V"].apply(get_curve)
df_merged["Bessell_R"] = df_merged["Bessell_R"].apply(get_curve)
df_merged["Bessell_I"] = df_merged["Bessell_I"].apply(get_curve)
df_merged["Bessell_B"] = df_merged["Bessell_B"].apply(get_curve)
df_merged["SLOAN_u"] = df_merged["SLOAN_u"].apply(get_curve)
df_merged["SLOAN_g"] = df_merged["SLOAN_g"].apply(get_curve)
df_merged["SLOAN_r"] = df_merged["SLOAN_r"].apply(get_curve)
df_merged["SLOAN_i"] = df_merged["SLOAN_i"].apply(get_curve)
df_merged["SLOAN_z"] = df_merged["SLOAN_z"].apply(get_curve)
df_merged["Kepler"] = df_merged["Kepler"].apply(get_curve)
df_merged["GaiaDR2"] = df_merged["GaiaDR2"].apply(get_curve)
df_merged["TESS"] = df_merged["TESS"].apply(get_curve)

In [25]:
# BLOK 8
# Vytvorenie tabulky. Jeden riadok = jedna svetelna krivka podla jedneho filtra.

df_final = pd.melt(df_merged, 
                   id_vars=["id","primary__t_eff", "secondary__t_eff", "inclination", "mass_ratio", 
                            "primary__surface_potential", "secondary__surface_potential"],
                   var_name="filter", value_name="curve")
df_final.describe()

,id,primary__t_eff,secondary__t_eff,inclination,mass_ratio,primary__surface_potential,secondary__surface_potential
count,1.300000e+06,1.300000e+06,1.300000e+06,1.300000e+06,1.300000e+06,1.300000e+06,1.300000e+06
mean,9.642836e+06,2.234418e+04,1.015223e+04,1.375013e+00,1.754530e+00,1.796074e+01,1.687004e+01
std,5.609623e+06,1.162822e+04,6.390062e+03,1.638420e-01,2.155654e+00,2.744355e+01,4.522165e+01
min,3.000000e+00,4.000000e+03,4.000000e+03,7.377260e-01,1.000000e-01,2.019959e+00,2.039986e+00
25%,4.691029e+06,1.200000e+04,6.000000e+03,1.267385e+00,6.000000e-01,4.984095e+00,4.380409e+00
50%,1.003546e+07,2.000000e+04,8.000000e+03,1.413717e+00,1.000000e+00,8.100800e+00,6.897109e+00
75%,1.439198e+07,3.000000e+04,1.200000e+04,1.509858e+00,2.000000e+00,1.589986e+01,1.329465e+01
max,1.862873e+07,4.500000e+04,4.500000e+04,1.570796e+00,1.000000e+01,1.100001e+02,9.965005e+02


In [28]:
# BLOK 9
# Uzavretie pripojenia na databazu. Ulozenie dat v podobe tabulky do suboru .pkl.

conn.close()
df_final.to_pickle("det_curves.pkl")

In [35]:
# BLOK 10
# Vytvorenie vzorky 500000 kriviek a 50000 pre ucely predikcie knn a svr

data_sample_knn=df_final.sample(n=500000, random_state=1234)
data_sample_svr = df_final.sample(n=40000, random_state=1234)

data_sample_knn.head()

,id,primary__t_eff,secondary__t_eff,inclination,mass_ratio,primary__surface_potential,secondary__surface_potential,filter,curve
647973,8747576,20000,14000,1.140601,0.5,3.027725,11.250627,SLOAN_g,"[0.9046986069364825, 0.9047564022519874, 0.904..."
1109932,2151919,40000,16000,1.570796,0.2,11.315166,5.400252,GaiaDR2,"[0.6924966300408725, 0.6934083009571762, 0.696..."
731380,6282803,35000,9000,1.548615,2.5,22.501256,14.992313,SLOAN_r,"[0.4460020548895491, 0.446035965957577, 0.4461..."
692703,18140128,8000,4000,1.375898,1.0,8.700863,5.032643,SLOAN_g,"[0.30110954686066566, 0.30399295740341686, 0.3..."
1145336,8511301,45000,9000,1.414878,2.0,13.115210,7.483131,GaiaDR2,"[0.22635886049323364, 0.2263575579813117, 0.22..."


In [36]:
# BLOK 11
# Ulozenie vzoriek v podobe tabulky do suboru .pkl

data_sample_knn.to_pickle("det_curves_samples_knn.pkl")
data_sample_svr.to_pickle("det_curves_samples_svr.pkl")